In [1]:
# Import dependecies
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
# Note that encoder.inverse_transform(df['popularity_encoded']) will return the original
# string values
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
import tensorflow as tf
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [2]:
# Import data for pre-processing
df = pd.read_csv('merged_dataframe.csv')
df.head(10)

,id,imdb_id,popularity,budget,revenue,cast,homepage,director,tagline,keywords,...,release_year,budget_adj,revenue_adj,film,year_film,year_ceremony,ceremony,category,name,winner
0,135397,tt0369610,32.985763,150000000,1513528810,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,monster|dna|tyrannosaurus rex|velociraptor|island,...,2015,1.379999e+08,1.392446e+09,Jurassic World,NaN,NaN,NaN,NaN,NaN,NaN
1,76341,tt1392190,28.419936,150000000,378436354,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,future|chase|post-apocalyptic|dystopia|australia,...,2015,1.379999e+08,3.481613e+08,Mad Max: Fury Road,2015.0,2016.0,88.0,CINEMATOGRAPHY,John Seale,False
2,262500,tt2908446,13.112507,110000000,295238201,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,based on novel|revolution|dystopia|sequel|dyst...,...,2015,1.012000e+08,2.716190e+08,Insurgent,NaN,NaN,NaN,NaN,NaN,NaN
3,140607,tt2488496,11.173104,200000000,2068178225,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,android|spaceship|jedi|space opera|3d,...,2015,1.839999e+08,1.902723e+09,Star Wars: The Force Awakens,2015.0,2016.0,88.0,FILM EDITING,Maryann Brandon and Mary Jo Markey,False
4,168259,tt2820852,9.335014,190000000,1506249360,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,car race|speed|revenge|suspense|car,...,2015,1.747999e+08,1.385749e+09,Furious 7,NaN,NaN,NaN,NaN,NaN,NaN
5,281957,tt1663202,9.110700,135000000,532950503,Leonardo DiCaprio|Tom Hardy|Will Poulter|Domhn...,http://www.foxmovies.com/movies/the-revenant,Alejandro GonzÃ¡lez IÃ±Ã¡rritu,"(n. One who has returned, as if from the dead.)",father-son relationship|rape|based on novel|mo...,...,2015,1.241999e+08,4.903142e+08,The Revenant,2015.0,2016.0,88.0,ACTOR IN A LEADING ROLE,Leonardo DiCaprio,True
6,87101,tt1340138,8.654359,155000000,440603537,Arnold Schwarzenegger|Jason Clarke|Emilia Clar...,http://www.terminatormovie.com/,Alan Taylor,Reset the future,saving the world|artificial intelligence|cybor...,...,2015,1.425999e+08,4.053551e+08,Terminator Genisys,NaN,NaN,NaN,NaN,NaN,NaN
7,286217,tt3659388,7.667400,108000000,595380321,Matt Damon|Jessica Chastain|Kristen Wiig|Jeff ...,http://www.foxmovies.com/movies/the-martian,Ridley Scott,Bring Him Home,based on novel|mars|nasa|isolation|botanist,...,2015,9.935996e+07,5.477497e+08,The Martian,2015.0,2016.0,88.0,ACTOR IN A LEADING ROLE,Matt Damon,False
8,211672,tt2293640,7.404165,74000000,1156730962,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,http://www.minionsmovie.com/,Kyle Balda|Pierre Coffin,"Before Gru, they had a history of bad bosses",assistant|aftercreditsstinger|duringcreditssti...,...,2015,6.807997e+07,1.064192e+09,Minions,NaN,NaN,NaN,NaN,NaN,NaN
9,150540,tt2096673,6.326804,175000000,853708609,Amy Poehler|Phyllis Smith|Richard Kind|Bill Ha...,http://movies.disney.com/inside-out,Pete Docter,Meet the little voices inside your head.,dream|cartoon|imaginary friend|animation|kid,...,2015,1.609999e+08,7.854116e+08,Inside Out,2015.0,2016.0,88.0,ANIMATED FEATURE FILM,Pete Docter and Jonas Rivera,True


In [3]:
df['profit_adj'] = df['revenue_adj'] - df['budget_adj']
df['profit_adj']

0        1.254446e+09
1        2.101614e+08
2        1.704191e+08
3        1.718723e+09
4        1.210949e+09
             ...     
10541    0.000000e+00
10542    0.000000e+00
10543    0.000000e+00
10544    0.000000e+00
10545   -1.276423e+05
Name: profit_adj, Length: 10546, dtype: float64

In [4]:
# Drop irrelevant or redundant columns
df = df.drop(['id', 'imdb_id', 'budget', 'revenue', 'homepage', 'tagline',
              'overview', 'year_film', 'year_ceremony', 'ceremony', 'name', 'film', 'keywords',
              'release_year', 'category', 'vote_count', 'vote_average'], axis = 1)
df.head(10)

,popularity,cast,director,runtime,genres,production_companies,release_date,budget_adj,revenue_adj,winner,profit_adj
0,32.985763,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,1.379999e+08,1.392446e+09,NaN,1.254446e+09
1,28.419936,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,1.379999e+08,3.481613e+08,False,2.101614e+08
2,13.112507,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,1.012000e+08,2.716190e+08,NaN,1.704191e+08
3,11.173104,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,1.839999e+08,1.902723e+09,False,1.718723e+09
4,9.335014,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,1.747999e+08,1.385749e+09,NaN,1.210949e+09
5,9.110700,Leonardo DiCaprio|Tom Hardy|Will Poulter|Domhn...,Alejandro GonzÃ¡lez IÃ±Ã¡rritu,156,Western|Drama|Adventure|Thriller,Regency Enterprises|Appian Way|CatchPlay|Anony...,12/25/2015,1.241999e+08,4.903142e+08,True,3.661143e+08
6,8.654359,Arnold Schwarzenegger|Jason Clarke|Emilia Clar...,Alan Taylor,125,Science Fiction|Action|Thriller|Adventure,Paramount Pictures|Skydance Productions,6/23/2015,1.425999e+08,4.053551e+08,NaN,2.627551e+08
7,7.667400,Matt Damon|Jessica Chastain|Kristen Wiig|Jeff ...,Ridley Scott,141,Drama|Adventure|Science Fiction,Twentieth Century Fox Film Corporation|Scott F...,9/30/2015,9.935996e+07,5.477497e+08,False,4.483897e+08
8,7.404165,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda|Pierre Coffin,91,Family|Animation|Adventure|Comedy,Universal Pictures|Illumination Entertainment,6/17/2015,6.807997e+07,1.064192e+09,NaN,9.961120e+08
9,6.326804,Amy Poehler|Phyllis Smith|Richard Kind|Bill Ha...,Pete Docter,94,Comedy|Animation|Family,Walt Disney Pictures|Pixar Animation Studios|W...,6/9/2015,1.609999e+08,7.854116e+08,True,6.244116e+08


In [5]:
print(df['release_date'].dtype)

object


In [6]:
# release_month[0:10]
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_month'] = df['release_date'].dt.month
df['release_month']

0         6
1         5
2         3
3        12
4         4
         ..
10541     6
10542    12
10543     1
10544    11
10545    11
Name: release_month, Length: 10546, dtype: int64

In [7]:
df = df.drop('release_date', axis = 1)
df.head(10)

,popularity,cast,director,runtime,genres,production_companies,budget_adj,revenue_adj,winner,profit_adj,release_month
0,32.985763,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,1.379999e+08,1.392446e+09,NaN,1.254446e+09,6
1,28.419936,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,1.379999e+08,3.481613e+08,False,2.101614e+08,5
2,13.112507,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,1.012000e+08,2.716190e+08,NaN,1.704191e+08,3
3,11.173104,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,1.839999e+08,1.902723e+09,False,1.718723e+09,12
4,9.335014,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,1.747999e+08,1.385749e+09,NaN,1.210949e+09,4
5,9.110700,Leonardo DiCaprio|Tom Hardy|Will Poulter|Domhn...,Alejandro GonzÃ¡lez IÃ±Ã¡rritu,156,Western|Drama|Adventure|Thriller,Regency Enterprises|Appian Way|CatchPlay|Anony...,1.241999e+08,4.903142e+08,True,3.661143e+08,12
6,8.654359,Arnold Schwarzenegger|Jason Clarke|Emilia Clar...,Alan Taylor,125,Science Fiction|Action|Thriller|Adventure,Paramount Pictures|Skydance Productions,1.425999e+08,4.053551e+08,NaN,2.627551e+08,6
7,7.667400,Matt Damon|Jessica Chastain|Kristen Wiig|Jeff ...,Ridley Scott,141,Drama|Adventure|Science Fiction,Twentieth Century Fox Film Corporation|Scott F...,9.935996e+07,5.477497e+08,False,4.483897e+08,9
8,7.404165,Sandra Bullock|Jon Hamm|Michael Keaton|Allison...,Kyle Balda|Pierre Coffin,91,Family|Animation|Adventure|Comedy,Universal Pictures|Illumination Entertainment,6.807997e+07,1.064192e+09,NaN,9.961120e+08,6
9,6.326804,Amy Poehler|Phyllis Smith|Richard Kind|Bill Ha...,Pete Docter,94,Comedy|Animation|Family,Walt Disney Pictures|Pixar Animation Studios|W...,1.609999e+08,7.854116e+08,True,6.244116e+08,6


In [8]:
df['director']

0           Colin Trevorrow
1             George Miller
2          Robert Schwentke
3               J.J. Abrams
4                 James Wan
                ...        
10541           Bruce Brown
10542    John Frankenheimer
10543        Eldar Ryazanov
10544           Woody Allen
10545      Harold P. Warren
Name: director, Length: 10546, dtype: object

In [9]:
df['cast'][0]

"Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vincent D'Onofrio|Nick Robinson"

In [10]:
type(df['cast'][0])

str

In [11]:
df['cast'] = df['cast'].str.split("|")
df['cast'][0]

['Chris Pratt',
 'Bryce Dallas Howard',
 'Irrfan Khan',
 "Vincent D'Onofrio",
 'Nick Robinson']

In [12]:
df['director'] = df['director'].str.split("|")
df['director'][0]

['Colin Trevorrow']

In [13]:
df['genres'] = df['genres'].str.split("|")
df['genres'][0]

['Action', 'Adventure', 'Science Fiction', 'Thriller']

In [14]:
df['production_companies'] = df['production_companies'].str.split("|")
df['production_companies'][0]

['Universal Studios',
 'Amblin Entertainment',
 'Legendary Pictures',
 'Fuji Television Network',
 'Dentsu']

In [15]:
df.shape

(10546, 11)

In [16]:
# Because some of the column values are lists of strings, rather than string or numeric types, 
# we separate the column and create a stacked Pandas Series object. This is then converted to a 
# dataframe.
new_df = df.drop('cast', axis = 1).join(
             pd.get_dummies(
             pd.DataFrame(df.cast.tolist(), df.index).stack(),
             prefix = 'cast_').astype(int).groupby(level=0).sum()
        )
new_df.head(10)

,popularity,director,runtime,genres,production_companies,budget_adj,revenue_adj,winner,profit_adj,release_month,...,cast__Ã‰milie Dequenne,cast__Ã‰ric Caravaca,cast__Ã‰va IgÃ³,cast__Ã€lex BrendemÃ¼hl,cast__Ã€lex Monner,cast__Äá»— Thá»‹ Háº£i Yáº¿n,cast__Å arÅ«nas Bartas,cast__Åžafak Sezer,cast__à¸ªà¸£à¸žà¸‡à¸©à¹Œ à¸Šà¸²à¸•à¸£à¸µ,cast__é‚±æ·‘è²ž
0,32.985763,[Colin Trevorrow],124,"[Action, Adventure, Science Fiction, Thriller]","[Universal Studios, Amblin Entertainment, Lege...",1.379999e+08,1.392446e+09,NaN,1.254446e+09,6,...,0,0,0,0,0,0,0,0,0,0
1,28.419936,[George Miller],120,"[Action, Adventure, Science Fiction, Thriller]","[Village Roadshow Pictures, Kennedy Miller Pro...",1.379999e+08,3.481613e+08,False,2.101614e+08,5,...,0,0,0,0,0,0,0,0,0,0
2,13.112507,[Robert Schwentke],119,"[Adventure, Science Fiction, Thriller]","[Summit Entertainment, Mandeville Films, Red W...",1.012000e+08,2.716190e+08,NaN,1.704191e+08,3,...,0,0,0,0,0,0,0,0,0,0
3,11.173104,[J.J. Abrams],136,"[Action, Adventure, Science Fiction, Fantasy]","[Lucasfilm, Truenorth Productions, Bad Robot]",1.839999e+08,1.902723e+09,False,1.718723e+09,12,...,0,0,0,0,0,0,0,0,0,0
4,9.335014,[James Wan],137,"[Action, Crime, Thriller]","[Universal Pictures, Original Film, Media Righ...",1.747999e+08,1.385749e+09,NaN,1.210949e+09,4,...,0,0,0,0,0,0,0,0,0,0
5,9.110700,[Alejandro GonzÃ¡lez IÃ±Ã¡rritu],156,"[Western, Drama, Adventure, Thriller]","[Regency Enterprises, Appian Way, CatchPlay, A...",1.241999e+08,4.903142e+08,True,3.661143e+08,12,...,0,0,0,0,0,0,0,0,0,0
6,8.654359,[Alan Taylor],125,"[Science Fiction, Action, Thriller, Adventure]","[Paramount Pictures, Skydance Productions]",1.425999e+08,4.053551e+08,NaN,2.627551e+08,6,...,0,0,0,0,0,0,0,0,0,0
7,7.667400,[Ridley Scott],141,"[Drama, Adventure, Science Fiction]","[Twentieth Century Fox Film Corporation, Scott...",9.935996e+07,5.477497e+08,False,4.483897e+08,9,...,0,0,0,0,0,0,0,0,0,0
8,7.404165,"[Kyle Balda, Pierre Coffin]",91,"[Family, Animation, Adventure, Comedy]","[Universal Pictures, Illumination Entertainment]",6.807997e+07,1.064192e+09,NaN,9.961120e+08,6,...,0,0,0,0,0,0,0,0,0,0
9,6.326804,[Pete Docter],94,"[Comedy, Animation, Family]","[Walt Disney Pictures, Pixar Animation Studios...",1.609999e+08,7.854116e+08,True,6.244116e+08,6,...,0,0,0,0,0,0,0,0,0,0


In [17]:
new_df.shape

(10546, 18648)

In [18]:
new_df = new_df.drop('genres', axis = 1).join(
             pd.get_dummies(
             pd.DataFrame(df.genres.tolist(), df.index).stack(),
             prefix = 'genre_').astype(int).groupby(level=0).sum()
        )
new_df.head(10)

,popularity,director,runtime,production_companies,budget_adj,revenue_adj,winner,profit_adj,release_month,cast__ Larry Mullen Jr.,...,genre__Horror,genre__Music,genre__Mystery,genre__None,genre__Romance,genre__Science Fiction,genre__TV Movie,genre__Thriller,genre__War,genre__Western
0,32.985763,[Colin Trevorrow],124,"[Universal Studios, Amblin Entertainment, Lege...",1.379999e+08,1.392446e+09,NaN,1.254446e+09,6,0,...,0,0,0,0,0,1,0,1,0,0
1,28.419936,[George Miller],120,"[Village Roadshow Pictures, Kennedy Miller Pro...",1.379999e+08,3.481613e+08,False,2.101614e+08,5,0,...,0,0,0,0,0,1,0,1,0,0
2,13.112507,[Robert Schwentke],119,"[Summit Entertainment, Mandeville Films, Red W...",1.012000e+08,2.716190e+08,NaN,1.704191e+08,3,0,...,0,0,0,0,0,1,0,1,0,0
3,11.173104,[J.J. Abrams],136,"[Lucasfilm, Truenorth Productions, Bad Robot]",1.839999e+08,1.902723e+09,False,1.718723e+09,12,0,...,0,0,0,0,0,1,0,0,0,0
4,9.335014,[James Wan],137,"[Universal Pictures, Original Film, Media Righ...",1.747999e+08,1.385749e+09,NaN,1.210949e+09,4,0,...,0,0,0,0,0,0,0,1,0,0
5,9.110700,[Alejandro GonzÃ¡lez IÃ±Ã¡rritu],156,"[Regency Enterprises, Appian Way, CatchPlay, A...",1.241999e+08,4.903142e+08,True,3.661143e+08,12,0,...,0,0,0,0,0,0,0,1,0,1
6,8.654359,[Alan Taylor],125,"[Paramount Pictures, Skydance Productions]",1.425999e+08,4.053551e+08,NaN,2.627551e+08,6,0,...,0,0,0,0,0,1,0,1,0,0
7,7.667400,[Ridley Scott],141,"[Twentieth Century Fox Film Corporation, Scott...",9.935996e+07,5.477497e+08,False,4.483897e+08,9,0,...,0,0,0,0,0,1,0,0,0,0
8,7.404165,"[Kyle Balda, Pierre Coffin]",91,"[Universal Pictures, Illumination Entertainment]",6.807997e+07,1.064192e+09,NaN,9.961120e+08,6,0,...,0,0,0,0,0,0,0,0,0,0
9,6.326804,[Pete Docter],94,"[Walt Disney Pictures, Pixar Animation Studios...",1.609999e+08,7.854116e+08,True,6.244116e+08,6,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
new_df.shape

(10546, 18668)

In [20]:
# new_df = new_df.drop('director', axis = 1).join(
#              pd.get_dummies(
#              pd.DataFrame(df.director.tolist(), df.index).stack()
#              ).astype(int).groupby(level=0).sum()
#         )
director_dummies = pd.get_dummies(new_df['director'].apply(pd.Series).stack()).groupby(level=0).sum()
director_dummies.columns = [f"{col}_director" for col in director_dummies.columns]

# concatenate the one-hot encoded columns back into the original dataframe
new_df = pd.concat([new_df.drop(['director'], axis=1), director_dummies], axis=1)
new_df.head(10)

,popularity,runtime,production_companies,budget_adj,revenue_adj,winner,profit_adj,release_month,cast__ Larry Mullen Jr.,cast__ Nicole MuÃ±oz,...,Zoya Akhtar_director,ÃdÃ¡m CsÃ¡szi_director,Ãlex de la Iglesia_director,Ãlvaro de ArmiÃ±Ã¡n_director,Ã–zgÃ¼r Yildirim_director,Ã‡aÄŸan Irmak_director,Ã‰mile Gaudreault_director,Ã‰ric Besnard_director,Ã‰tienne Chatiliez_director,Ã€lex Pastor_director
0,32.985763,124,"[Universal Studios, Amblin Entertainment, Lege...",1.379999e+08,1.392446e+09,NaN,1.254446e+09,6,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,28.419936,120,"[Village Roadshow Pictures, Kennedy Miller Pro...",1.379999e+08,3.481613e+08,False,2.101614e+08,5,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13.112507,119,"[Summit Entertainment, Mandeville Films, Red W...",1.012000e+08,2.716190e+08,NaN,1.704191e+08,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11.173104,136,"[Lucasfilm, Truenorth Productions, Bad Robot]",1.839999e+08,1.902723e+09,False,1.718723e+09,12,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9.335014,137,"[Universal Pictures, Original Film, Media Righ...",1.747999e+08,1.385749e+09,NaN,1.210949e+09,4,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,9.110700,156,"[Regency Enterprises, Appian Way, CatchPlay, A...",1.241999e+08,4.903142e+08,True,3.661143e+08,12,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,8.654359,125,"[Paramount Pictures, Skydance Productions]",1.425999e+08,4.053551e+08,NaN,2.627551e+08,6,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7.667400,141,"[Twentieth Century Fox Film Corporation, Scott...",9.935996e+07,5.477497e+08,False,4.483897e+08,9,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,7.404165,91,"[Universal Pictures, Illumination Entertainment]",6.807997e+07,1.064192e+09,NaN,9.961120e+08,6,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,6.326804,94,"[Walt Disney Pictures, Pixar Animation Studios...",1.609999e+08,7.854116e+08,True,6.244116e+08,6,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
new_df.shape

(10546, 23957)

In [22]:
new_df = new_df.drop('production_companies', axis = 1).join(
             pd.get_dummies(
             pd.DataFrame(df.production_companies.tolist(), df.index).stack(),
             prefix = 'production_companies_').astype(int).groupby(level=0).sum()
        )
new_df.head(10)

,popularity,runtime,budget_adj,revenue_adj,winner,profit_adj,release_month,cast__ Larry Mullen Jr.,cast__ Nicole MuÃ±oz,cast__ Therese Giehse,...,production_companies__platinum studios,production_companies__teamWorx Produktion fÃ¼r Kino und Fernsehen GmbH,production_companies__the filmgroup,production_companies__thefyzz,production_companies__thinkfilm,production_companies__uFilm,production_companies__uFund,production_companies__verture Films,production_companies__Ã–sterreichischer Rundfunk (ORF),production_companies__í”„ë¡œë•ì…˜M
0,32.985763,124,1.379999e+08,1.392446e+09,NaN,1.254446e+09,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,28.419936,120,1.379999e+08,3.481613e+08,False,2.101614e+08,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,13.112507,119,1.012000e+08,2.716190e+08,NaN,1.704191e+08,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11.173104,136,1.839999e+08,1.902723e+09,False,1.718723e+09,12,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9.335014,137,1.747999e+08,1.385749e+09,NaN,1.210949e+09,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,9.110700,156,1.241999e+08,4.903142e+08,True,3.661143e+08,12,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,8.654359,125,1.425999e+08,4.053551e+08,NaN,2.627551e+08,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7.667400,141,9.935996e+07,5.477497e+08,False,4.483897e+08,9,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,7.404165,91,6.807997e+07,1.064192e+09,NaN,9.961120e+08,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,6.326804,94,1.609999e+08,7.854116e+08,True,6.244116e+08,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
new_df.shape

(10546, 31676)

In [24]:
new_df['winner']

0          NaN
1        False
2          NaN
3        False
4          NaN
         ...  
10541      NaN
10542     True
10543      NaN
10544      NaN
10545      NaN
Name: winner, Length: 10546, dtype: object

In [25]:
# Convert null values to False
new_df['winner'] = new_df['winner'].fillna(0).astype(float)

In [26]:
# Convert boolean values to binary values
new_df['winner'] = new_df['winner'].astype(float)

In [27]:
new_df['winner']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
10541    0.0
10542    1.0
10543    0.0
10544    0.0
10545    0.0
Name: winner, Length: 10546, dtype: float64

In [28]:
new_df = new_df.fillna(0)

In [29]:
new_df.isnull().sum()

popularity                                                0
runtime                                                   0
budget_adj                                                0
revenue_adj                                               0
winner                                                    0
                                                         ..
production_companies__uFilm                               0
production_companies__uFund                               0
production_companies__verture Films                       0
production_companies__Ã–sterreichischer Rundfunk (ORF)    0
production_companies__í”„ë¡œë•ì…˜M                        0
Length: 31676, dtype: int64

In [30]:
y = new_df['winner']
X = new_df.copy().drop(columns='winner')
X.head(10)

,popularity,runtime,budget_adj,revenue_adj,profit_adj,release_month,cast__ Larry Mullen Jr.,cast__ Nicole MuÃ±oz,cast__ Therese Giehse,"cast__""Weird Al"" Yankovic",...,production_companies__platinum studios,production_companies__teamWorx Produktion fÃ¼r Kino und Fernsehen GmbH,production_companies__the filmgroup,production_companies__thefyzz,production_companies__thinkfilm,production_companies__uFilm,production_companies__uFund,production_companies__verture Films,production_companies__Ã–sterreichischer Rundfunk (ORF),production_companies__í”„ë¡œë•ì…˜M
0,32.985763,124,1.379999e+08,1.392446e+09,1.254446e+09,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,28.419936,120,1.379999e+08,3.481613e+08,2.101614e+08,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,13.112507,119,1.012000e+08,2.716190e+08,1.704191e+08,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11.173104,136,1.839999e+08,1.902723e+09,1.718723e+09,12,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9.335014,137,1.747999e+08,1.385749e+09,1.210949e+09,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,9.110700,156,1.241999e+08,4.903142e+08,3.661143e+08,12,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,8.654359,125,1.425999e+08,4.053551e+08,2.627551e+08,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7.667400,141,9.935996e+07,5.477497e+08,4.483897e+08,9,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,7.404165,91,6.807997e+07,1.064192e+09,9.961120e+08,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,6.326804,94,1.609999e+08,7.854116e+08,6.244116e+08,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# Check the balance of our target values
y.value_counts()

0.0    10314
1.0      232
Name: winner, dtype: int64

In [32]:
X.shape

(10546, 31675)

In [33]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [34]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [35]:
# Because the data are so heavily imbalanced toward the number
# of films that do not win Oscar awards, we will use SMOTE resampling
# as a way to effectively train the model.
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train_scaled, y_train
)
Counter(y_resampled)

Counter({0.0: 7740, 1.0: 7740})

In [36]:
nn_dim = X.shape[1]
nn_dim

31675

In [37]:
# Create the architecture for the neural network
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=1500, activation="relu", input_dim=nn_dim))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=500, activation = "relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = 50, activation = "relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units = 5, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1500)              47514000  
                                                                 
 dense_1 (Dense)             (None, 500)               750500    
                                                                 
 dense_2 (Dense)             (None, 50)                25050     
                                                                 
 dense_3 (Dense)             (None, 5)                 255       
                                                                 
 dense_4 (Dense)             (None, 1)                 6         
                                                                 
Total params: 48,289,811
Trainable params: 48,289,811
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [39]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=8)

Epoch 1/8
248/248 [==============================] - 72s 284ms/step - loss: 0.1230 - accuracy: 0.9765
Epoch 2/8
248/248 [==============================] - 72s 289ms/step - loss: 0.0302 - accuracy: 0.9890
Epoch 3/8
248/248 [==============================] - 72s 291ms/step - loss: 0.0079 - accuracy: 0.9976
Epoch 4/8
248/248 [==============================] - 72s 291ms/step - loss: 0.0075 - accuracy: 0.9966
Epoch 5/8
248/248 [==============================] - 71s 288ms/step - loss: 0.0027 - accuracy: 0.9997
Epoch 6/8
248/248 [==============================] - 71s 285ms/step - loss: 0.0039 - accuracy: 0.9986
Epoch 7/8
248/248 [==============================] - 72s 292ms/step - loss: 0.0025 - accuracy: 0.9987
Epoch 8/8
248/248 [==============================] - 71s 286ms/step - loss: 0.0020 - accuracy: 0.9995


In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

83/83 - 3s - loss: 0.3670 - accuracy: 0.9408 - 3s/epoch - 36ms/step


In [41]:
y_test.value_counts()

0.0    2574
1.0      63
Name: winner, dtype: int64

In [42]:
y_pred = np.argmax(nn.predict(X_test_scaled), axis=1)
#ravel(nn.predict_classes(X_test_scaled))
pd.Series(y_pred).value_counts()

83/83 [==============================] - 2s 28ms/step


0    2637
dtype: int64

In [43]:
from sklearn.metrics import confusion_matrix

y_pred = nn.predict(X_test_scaled)
confusion = confusion_matrix(y_test, y_pred)
confusion

# [[true negative  false positive]
#  [false negative true positive]]

83/83 [==============================] - 3s 31ms/step


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
# Unable to produce a confusion matrix due to the following error:
# ValueError: Classification metrics can't handle a mix of binary and continuous targets
# This error also prevents the calculation of precision and recall.

In [46]:
# Now that we have the results from the neural network, train and evaluate a random forest model for comparision.
# Resample the training data with the BalancedRandomForestClassifier
balanced_rf_classifier = BalancedRandomForestClassifier(random_state = 1)
balanced_rf_classifier.fit(X_train, y_train)

# Fit the Logistic Regression model to the resampled training data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [47]:
# Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
# List the features sorted in descending order by feature importance
# Get the feature importances
importances = balanced_rf_classifier.feature_importances_

# Sort the feature importances in descending order
feature_importances = sorted(zip(importances, X_train.columns), reverse=True)

# Print the feature importances
for feature_importance in feature_importances:
    print(feature_importance)